# Start

In [97]:
datadir = '../data';
wkdir = '../results';

mk_cd_dir(wkdir, false);
%imatlab_export_fig('print-png')

% loocv inputs

prepare_loocv = true

demo_loocv = true

demo_loocv_number_or_list = 'number' % 'number' or 'list'

demo_loocv_number_cmpds = 2

demo_loocv_list_cmpds = {'BRD-K04804440','BRD-K01507359','BRD-K87202646','BRD-K59853741', 'BRD-K27302037'} % Ciprofloxacin, Rifampin, Isoniazid, Q203, Thioacetazone

results_subdir_prefix = 'loocv_pcls/leave_out_cmpd_'

loocv_save_fig = true % .png files of spectral clustering input and output

loocv_save_out = true % save tabular and gmt files for each MOA separately with the output of spectral clsutering


% general inputs

unique_kabx_cmpds_tbl_path = '../results/kabx_pert_ids_tbl_for_loocv.txt'

moa_figdir = 'verify_moas/figures'

moa_gctdir = 'verify_moas/gcts'

corr_for_pcls_savepath = 'sGR_for_pcls_pearson_corr'

corr_rank_for_pcls_savepath = 'sGR_for_pcls_pearson_corr_rank'

moa_gmt_savepath = 'moas.gmt'

save_fig = true % .png files of spectral clustering input and output

save_out = true % save tabular and gmt files for each MOA separately with the output of spectral clsutering


% Spectral Clustering inputs

random_seed = 0; % specified seed for initializing the random number generator, Matlab factory default is the Mersenne Twister generator with seed 0 (see spectral_clustering_for_pcls.m for additional information)

thrsh_rank = 20 % threshold for average pairwise rank of correlation across KABX to connect treatments as mutual nearest-neighbors

thrsh_factor = 1; % factor to multiply by thrsh_rank if dynamic_thrsh_per_moa is false; default is 1

dynamic_thrsh_per_moa = false % if true then threshold is round(log(size of MOA) * thrsh_rank), otherwise identical threshold for every MOA

k_type = 'k_med_gap_den' % eigengap heuristic to take for estimating number of K clusters: k_num_zero, k_num_zero_plus_one, k_med_gap_den, k_gap_den (see create_laplacian_matrix.m for additional information)


% outputs

if dynamic_thrsh_per_moa
    outdir_name = sprintf('clusters_spectral_clustering_thrsh_rank_le%dxlogmoasize_%s', thrsh_rank, k_type)
else
    outdir_name = sprintf('clusters_spectral_clustering_thrsh_rank_le%d_%s', thrsh_rank, k_type)
end

outdir = fullfile(wkdir, outdir_name)

clusters_tbl_incl_singletons_filename = 'clusters_spectral_clust_including_singletons.txt'

clusters_tbl_filename = 'clusters_spectral_clust.txt'

clusters_gmt_filename = 'clusters_spectral_clust.gmt'


prepare_loocv =

  logical

   1


demo_loocv =

  logical

   1


demo_loocv_number_or_list =

    'number'


demo_loocv_number_cmpds =

     2


demo_loocv_list_cmpds =

  1x5 cell array

  Columns 1 through 3

    {'BRD-K04804440'}    {'BRD-K01507359'}    {'BRD-K87202646'}

  Columns 4 through 5

    {'BRD-K59853741'}    {'BRD-K27302037'}


results_subdir_prefix =

    'loocv_pcls/leave_out_cmpd_'


loocv_save_fig =

  logical

   1


loocv_save_out =

  logical

   1


unique_kabx_cmpds_tbl_path =

    '../results/kabx_pert_ids_tbl_for_loocv.txt'


moa_figdir =

    'verify_moas/figures'


moa_gctdir =

    'verify_moas/gcts'


corr_for_pcls_savepath =

    'sGR_for_pcls_pearson_corr'


corr_rank_for_pcls_savepath =

    'sGR_for_pcls_pearson_corr_rank'


moa_gmt_savepath =

    'moas.gmt'


save_fig =

  logical

   1


save_out =

  logical

   1


thrsh_rank =

    20


dynamic_thrsh_per_moa =

  logical

   0


k_type =

    'k_med_gap_den'


outdir_name =

    'clusters_spect

In [98]:
switch k_type
case 'k_gap_den'
case 'k_med_gap_den'
case 'k_num_zero'
case 'k_num_zero_plus_one'
otherwise
    error('Unknown k_type; Only "k_gap_den", "k_med_gap_den", "k_num_zero_plus_one", or "k_num_zero" can be used')
end

## Header for selecting leftout compound directory to work in

In [17]:
loo_pert_id_idx = 1


loo_pert_id_idx =

     1



In [18]:
unique_kabx_cmpds_tbl = rtable('/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript_final/loocv_pcls/kabx_pert_ids_tbl_for_loocv.txt');

size(unique_kabx_cmpds_tbl)
headt(unique_kabx_cmpds_tbl)

Reading /idi/cgtb/abond/combine_screens_456/pcls_for_manuscript/loocv_pcls/kabx_pert_ids_tbl_for_loocv.txt


ans =

   434     2


ans =

  2x3 table

    idx          field                value      
    ___    _________________    _________________

     1     {'kabx_cmpd_idx'}    {[            1]}
     2     {'kabx_cmpd'    }    {'BRD-A02179977'}



In [19]:
this_loocv_cmpd = unique_kabx_cmpds_tbl.kabx_cmpd(unique_kabx_cmpds_tbl.kabx_cmpd_idx == loo_pert_id_idx)

this_loocv_cmpd_string = strjoin(this_loocv_cmpd)


this_loocv_cmpd =

  1x1 cell array

    {'BRD-A02179977'}


this_loocv_cmpd_string =

    'BRD-A02179977'



## End Header

In [2]:
datadir = '/idi/cgtb/morzech/idmp/combine_screens_456';
wkdir = ['/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript_final/loocv_pcls/', 'leave_out_cmpd_', this_loocv_cmpd_string]
%wkdir = fullfile(datadir,'pcls_remove_gsk_no_broad_sar_pearson');
outdir = fullfile(wkdir,'pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den_fast')

mk_cd_dir(outdir,false);
mk_cd_dir(wkdir, true);
imatlab_export_fig('print-png')


wkdir =

    '/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript'


outdir =

    '/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den'



In [16]:
% Ensure parallel pool is cleaned up at the end of the script or on error
cleanupObj = onCleanup(@() delete(gcp('nocreate')));

In [17]:
p = parpool(feature('numcores'),'IdleTimeout',120);

Starting parallel pool (parpool) using the 'local' profile ...
Connected to the parallel pool (number of workers: 2).


In [99]:
moas = parse_gmt(fullfile(wkdir,moa_gmt_savepath));

In [100]:
mk_cd_dir(outdir, false);
mk_cd_dir(wkdir, false);

# Run Spectral Clustering (and Plot Eigenvalues) for Each MOA

In [117]:
spec = {};
out_lap = {};

out = {};
out_nonsingle = {};
gmt = {};

num_moas_represented = 0;


for ii = 53:56
%parfor ii = 1:numel(moas)
%parfor ii = 1:4
    loop_progress(ii,numel(moas),1)
    
    % Set MoA class
    moa_class = moas(ii).head;
    out_lap(ii).moa_class = moa_class;
    disp(moa_class)
    
    %[strrep(strrep(moa_class,' ','_'),'/','-'),'_corr_n*.gctx']
    
    % Load gcts
    g = glob(fullfile(wkdir, moa_gctdir,[strrep(strrep(moa_class,' ','_'),'/','-'),'_corr_n*.gctx']))
    
    try
        c = parse_gctx(g{1});
    catch ME
        disp(sprintf('Error: %s correlation gctx file not found', moa_class))
        disp(ME)
    end
    
    g = glob(fullfile(wkdir, moa_gctdir,[strrep(strrep(moa_class,' ','_'),'/','-'),'_corr_rank_n*.gctx']));
    
    try
        cr = parse_gctx(g{1});
    catch ME
        disp(sprintf('Error: %s correlation rank gctx file not found', moa_class))
        disp(ME)
    end
    
    % Set threshold
    if dynamic_thrsh_per_moa
        thrsh = round(log(numel(cr.rid)) * thrsh_rank);
    else
        thrsh = thrsh_rank*thrsh_factor;
    end
    
    out_lap(ii).thrsh_rank = thrsh_rank;
    out_lap(ii).thrsh_factor = thrsh_factor;
    out_lap(ii).thrsh = thrsh;
    
    % Run spectral clustering
    try
        [spec(ii).tmp_out_nonsingle,spec(ii).tmp_out,spec(ii).tmp_gmt,L,out_lap(ii).k,en,den,out_lap(ii).k_gap_den,out_lap(ii).k_med_gap_den,out_lap(ii).k_num_zero_plus_one,out_lap(ii).k_num_zero] = spectral_clustering_for_pcls(c,cr,moa_class,thrsh,k_type,outdir,save_out,save_fig,random_seed);
        close all
    catch ME
        disp('Error from spectral clustering')
        disp(ME)
    end
    
end

out_lap = struct2table(out_lap);
wtable(out_lap,fullfile(outdir,'k_values.txt'))

disp('All done')

for ii = 1:numel(moas)

    moa_class = moas(ii).head

    if size(spec(ii).tmp_gmt, 1) > 0
    
        fields = unique([spec(ii).tmp_out_nonsingle.Properties.VariableNames,spec(ii).tmp_out.Properties.VariableNames]);
    
        for jj = 1:numel(fields)
            try
                spec(ii).tmp_out_nonsingle.(fields{jj}) = any2str(spec(ii).tmp_out_nonsingle.(fields{jj}));
            catch ME
                true;
            end

            try
                spec(ii).tmp_out.(fields{jj}) = any2str(spec(ii).tmp_out.(fields{jj}));
            catch ME
                true;
            end
        end
    
        num_moas_represented = num_moas_represented + 1;
    else
        disp(moa_class)
        disp("NOT INCLUDED - NO NON-SINGLETON CLUSTERS")
    end
    
end


disp(sprintf('Number of MOAs with non-singleton clusters: %d', num_moas_represented))
disp(sprintf('Total number of MOAs: %d', numel(moas)))

disp(sprintf('Proportion of MOAs with non-singleton clusters: %f', num_moas_represented / numel(moas)))

53/71
RpoB

g =

  1x1 cell array

    {'../results/verify_moas/gcts/RpoB_corr_n270x270.gctx'}

Reading ../results/verify_moas/gcts/RpoB_corr_n270x270.gctx [270x270]
Done [0.07 s].
Reading ../results/verify_moas/gcts/RpoB_corr_rank_n270x270.gctx [270x270]
Done [0.07 s].
54/71
ThyAX

g =

  1x1 cell array

    {'../results/verify_moas/gcts/ThyAX_corr_n308x308.gctx'}

Reading ../results/verify_moas/gcts/ThyAX_corr_n308x308.gctx [308x308]
Done [0.08 s].
Reading ../results/verify_moas/gcts/ThyAX_corr_rank_n308x308.gctx [308x308]
Done [0.08 s].
55/71
TopA

g =

  1x1 cell array

    {'../results/verify_moas/gcts/TopA_corr_n500x500.gctx'}

Reading ../results/verify_moas/gcts/TopA_corr_n500x500.gctx [500x500]
Done [0.13 s].
Reading ../results/verify_moas/gcts/TopA_corr_rank_n500x500.gctx [500x500]
Done [0.11 s].
56/71
TrpAB

g =

  1x1 cell array

    {'../results/verify_moas/gcts/TrpAB_corr_n60x60.gctx'}

Reading ../results/verify_moas/gcts/TrpAB_corr_n60x60.gctx [60x60]
Done [0.03 s].
Error

In [114]:
g{1}

Index exceeds the number of array elements (0).


In [110]:
out = cat(1, spec.tmp_out);

out_nonsingle = cat(1, spec.tmp_out_nonsingle);

gmt = cat(1, spec.tmp_gmt);

height(out)

headt(out)

height(out_nonsingle)

headt(out_nonsingle)

sum([gmt.len]>2)
sum([gmt.len]<=2)

disp(sprintf('Number of clusters of size greater than 2: %d', sum([gmt.len]>2)))

disp(sprintf('Number of clusters of size 2: %d', sum([gmt.len]<=2)))

disp(sprintf('Number of clusters: %d', size(gmt, 1)))

Error using tabular/cat (line 16)
Cannot concatenate the table variable 'pert_id_pubchem_cid' because it is a cell in one table and a non-cell in another.


In [103]:
% Save the outputs
wtable(out,fullfile(outdir, clusters_tbl_incl_singletons_filename))
wtable(out_nonsingle,fullfile(outdir, clusters_tbl_filename))
mkgmt(fullfile(outdir, clusters_gmt_filename),gmt)

Error using wtable (line 22)
wtable> Structure "out" in the input is not a table


In [50]:
cat(1,gmt{:})


ans = 

  181x1 struct array with fields:

    head
    desc
    entry
    len



In [52]:
cat(1,spec.tmp_gmt{:})

Expected one output from a curly brace or dot indexing expression, but there were 4 results.


In [58]:
cat(1, spec.tmp_gmt)


ans = 

  181x1 struct array with fields:

    head
    desc
    entry
    len



In [45]:
size(spec(ii).tmp_gmt, 1)

size({}, 1)


ans =

    16


ans =

     0



## Plot eigenvalues

In [10]:
thrsh_factor = 1;
thrsh_rank = 20;
%k_type = 'k_num_zero';
k_type = 'k_med_gap_den';
save_fig = true;
save_out = false;


out_lap = {};
for ii = 1:numel(pcls)
    loop_progress(ii,numel(pcls),10)
    
    % Set MoA class
    moa_class = pcls(ii).head;
    out_lap(ii).moa_class = moa_class;
    
    % Load gcts
    g = glob(fullfile(wkdir,'verify_pcls','gcts',[strrep(strrep(moa_class,' ','_'),'/','-'),'_corr_n*.gct']));
    c = parse_gctx(g{1});
    g = glob(fullfile(wkdir,'verify_pcls','gcts',[strrep(strrep(moa_class,' ','_'),'/','-'),'_corr_rank_n*.gct']));
    cr = parse_gctx(g{1});
    
    % Set threshold
    thrsh = thrsh_rank*thrsh_factor;
    out_lap(ii).thrsh_factor = thrsh_factor;
    out_lap(ii).thrsh = thrsh;

    % Create adjacency matrix
    adj_mat = double(cr.mat<=thrsh);

    % Normalized Laplacian
    [L,out_lap(ii).k,en,den,out_lap(ii).k_gap_den,out_lap(ii).k_num_zero] = create_laplacian_edit_ab(adj_mat,'symmetric',save_fig,k_type);
    
    moa_class_str = strrep(strrep(moa_class,' ','_'),'/','-');
    saveas_png(gcf,outdir,[moa_class_str,'_eigenvalues.png'])
    close(gcf)
end
out_lap = struct2table(out_lap);
wtable(out_lap,fullfile(outdir,'k_values.txt'))
disp('All done')

1/111
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2016_series_(topoisomerase_I)_corr_n64x64.gct [64x64]
class:single
read:64/64
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2016_series_(topoisomerase_I)_corr_rank_n64x64.gct [64x64]
class:single
read:64/64
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2626_series_corr_n112x112.gct [112x112]
class:single
read:112/112
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2626_series_corr_rank_n112x112.gct [112x112]
class:single
read:112/112
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/5F-THF-antidote_to_folate_inhibitors_corr_n20x20.gct [20x20]
class:single
read:20/20
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/5F-THF-antidote_to_folate_inhibitors_corr_rank_n20x20.gct [20x20]
class:singl

## Run Spectral Clustering

In [64]:
thrsh_rank = 20;
thrsh_factor = 1;
%k_type = 'k_num_zero';
k_type = 'k_med_gap_den';
save_fig = true;
save_out = true;


parfor ii = 1:numel(pcls)
    loop_progress(ii,numel(pcls),1)
    
    % Set MoA class
    moa_class = pcls(ii).head;
    disp(moa_class)
    [strrep(strrep(moa_class,' ','_'),'/','-'),'_corr_n*.gct']
    % Load gcts
    g = glob(fullfile(wkdir,'verify_pcls','gcts',[strrep(strrep(moa_class,' ','_'),'/','-'),'_corr_n*.gct']))
    c = parse_gctx(g{1});
    g = glob(fullfile(wkdir,'verify_pcls','gcts',[strrep(strrep(moa_class,' ','_'),'/','-'),'_corr_rank_n*.gct']));
    cr = parse_gctx(g{1});
    
    % Set threshold
    thrsh = thrsh_rank*thrsh_factor;
    
    % Run spectral clustering
    try
        [spec(ii).tmp_out_nonsingle,spec(ii).tmp_out,spec(ii).tmp_gmt] = spectral_clust_for_pcls_fast(c,cr,moa_class,thrsh,k_type,outdir,save_out,save_fig);
        close all
    catch ME
        disp('Error from spectral clustering')
        disp(ME)
    end
end
disp('All done')

1/111
2016 series (topoisomerase I)

ans =

    '2016_series_(topoisomerase_I)_corr_n*.gct'

Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2016_series_(topoisomerase_I)_corr_n64x64.gct [64x64]
class:single
read:64/64
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2016_series_(topoisomerase_I)_corr_rank_n64x64.gct [64x64]
class:single
read:64/64
Done.
2/111
2626 series

ans =

    '2626_series_corr_n*.gct'

Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2626_series_corr_n112x112.gct [112x112]
class:single
read:112/112
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2626_series_corr_rank_n112x112.gct [112x112]
class:single
read:112/112
Done.
All done


In [10]:
spec_bak = spec;

## Combine tables

In [51]:
spec = {};
out = {};
out_nonsingle = {};
gmt = {};

num_moas_represented = 0;

for ii = 1:numel(pcls)
    loop_progress(ii,numel(pcls),10)
    
    moa_class = pcls(ii).head;
    moa_class_str = strrep(strrep(moa_class,' ','_'),'/','-');
    
    out_path = fullfile(outdir,[moa_class_str,'_spectral_clust.txt']);
    out_nonsingle_path = fullfile(outdir,[moa_class_str,'_spectral_clust_nonsingle.txt']);
    gmt_path = fullfile(outdir,[moa_class_str,'_spectral_clust.gmt']);
    
    if exist(gmt_path, 'file') == 2
        spec(ii).tmp_out_nonsingle = rtable(out_nonsingle_path);
        spec(ii).tmp_out = rtable(out_path);
        spec(ii).tmp_gmt = parse_gmt(gmt_path);
        
        fields = unique([spec(ii).tmp_out_nonsingle.Properties.VariableNames,spec(ii).tmp_out.Properties.VariableNames]);
    
        for jj = 1:numel(fields)
            try
                spec(ii).tmp_out_nonsingle.(fields{jj}) = any2str(spec(ii).tmp_out_nonsingle.(fields{jj}));
            catch ME
                true;
            end

            try
                spec(ii).tmp_out.(fields{jj}) = any2str(spec(ii).tmp_out.(fields{jj}));
            catch ME
                true;
            end
        end

        out_nonsingle = [out_nonsingle;spec(ii).tmp_out_nonsingle];
        out = [out;spec(ii).tmp_out];
        gmt = [gmt,spec(ii).tmp_gmt];
        
        num_moas_represented = num_moas_represented + 1;
        
    else
        disp(moa_class)
        disp("NOT INCLUDED - NO NON-SINGLETON PCLS")
        
    end
    
end

num_moas_represented
numel(pcls)

num_moas_represented / numel(pcls)

1/111


In [52]:
gmt = cat(1,gmt{:});

In [59]:
sum([gmt.len]>2)
sum([gmt.len]<=2)

size(gmt, 1)


ans =

     4


ans =

     3


ans =

     7



In [40]:
% Save the outputs
wtable(out,fullfile(outdir,'pcls_spectral_clust.txt'))
wtable(out_nonsingle,fullfile(outdir,'pcls_spectral_clust_nonsingle.txt'))
mkgmt(fullfile(outdir,'pcls_spectral_clust.gmt'),gmt)